## Importance Sampling Example

Author: Audrey Olivier 

Date: May 8, 2019

For importance sampling, the function must be written in a way that it can evaluate multiple samples at once.

In [ ]:
from UQpy.distributions import Uniform, JointIndependent
from UQpy.sampling import ImportanceSampling
import time
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def log_Rosenbrock(x, param):
    return (-(100 * (x[:, 1] - x[:, 0] ** 2) ** 2 + (1 - x[:, 0]) ** 2) / param)

In [ ]:
proposal = JointIndependent([Uniform(loc=-8, scale=16), Uniform(loc=-10, scale=60)])
print(proposal.get_parameters())

### Run IS

In [ ]:
t4 = time.time()
from UQpy.sampling.input_data import ISInput

is_input = ISInput(log_pdf_target=log_Rosenbrock, args_target=(20,), proposal=proposal)

w = ImportanceSampling(is_input=is_input, samples_number=10000)

t_IS = time.time() - t4
print(t_IS)

w.resample(samples_number=1000)
plt.plot(w.unweighted_samples[:, 0], w.unweighted_samples[:, 1], 'gs', alpha=0.2)
print(w.unweighted_samples.shape)
plt.legend(['IS'])
plt.show()

### Run IS by adding samples: call the run method in a loop (one can also look at diagnostics)

In [ ]:
t4 = time.time()
is_input1 = ISInput()
is_input1.log_pdf_target = log_Rosenbrock
is_input1.args_target = (20,)
is_input1.proposal = proposal

w = ImportanceSampling(is_input=is_input1)
for nsamples in [5000, 5000, 5000, 5000]:
    w.run(nsamples)
    print(w.samples.shape)
    # IS_diagnostics(weights=w.weights, graphics=False)
t_IS = time.time() - t4
print(t_IS)

w.resample(samples_number=1000)
plt.plot(w.unweighted_samples[:, 0], w.unweighted_samples[:, 1], 'gs', alpha=0.2)
plt.legend(['IS'])
plt.show()

Another example: sampling from a bivariate with copula dependence. Giving a random state enforces that results are the same for repeatability.

In [ ]:
from UQpy.distributions import Normal, Gumbel, JointCopula
dist_true = JointCopula(marginals=[Normal(), Normal()], copula=Gumbel(theta=2.))
proposal1 = JointIndependent(marginals=[Normal(), Normal()])

is_input2 = ISInput()
is_input2.log_pdf_target = dist_true.log_pdf
is_input2.proposal = proposal1
is_input2.random_state = 123

sampler = ImportanceSampling(is_input=is_input2, samples_number=500)
print(sampler.samples.shape)
print(sampler.weights.shape)
print(np.round(sampler.samples[-5:], 4))